In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pydicom
import imageio
import os
from glob import glob
import nibabel as nib
from tqdm import tqdm
import json
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

MRI

In [6]:

save_class_list1=['DWI']
class_list=['0100.Ischemic Stroke','0300.Stroke Control Group']
save_class_list=['Ischemic','Normal']
data_dir='../../data/stroke_2023/**/1.원천데이터/'
save_dir='../../data/stroke_mri_nii/images'

In [8]:

for k in range(len(class_list)):
    for j in range(len(save_class_list1)):
        data_list = glob(data_dir + class_list[k] + '/*MR Diffusion/*' + save_class_list1[j] + '*.dcm')

        # 환자 ID 별로 그룹화
        patient_dict = {}
        for dicom_path in data_list:
            file_name = os.path.basename(dicom_path)
            folder_split = file_name.split('_')
            patient_id = folder_split[0] + '_' + folder_split[1]
            if patient_id not in patient_dict:
                patient_dict[patient_id] = []
            patient_dict[patient_id].append(dicom_path)

        for patient_id, dicom_paths in tqdm(patient_dict.items(), desc=f"{class_list[k]}_{save_class_list1[j]}"):
            # 저장할 폴더
            folder_path = os.path.join(save_dir, save_class_list1[j])
            create_dir(folder_path)

            slices = []
            for dicom_file in sorted(dicom_paths):  # slice 순서 보존
                dcm = pydicom.dcmread(dicom_file)
                img = dcm.pixel_array.astype(np.float32)

                # RescaleSlope / Intercept 적용
                slope = float(getattr(dcm, 'RescaleSlope', 1))
                intercept = float(getattr(dcm, 'RescaleIntercept', 0))
                img = img * slope + intercept

                slices.append(img)

            volume = np.stack(slices, axis=0)  # (Z, H, W)

            # ✅ 정규화: -1 ~ 1
            vmin = np.percentile(volume, 1)
            vmax = np.percentile(volume, 99)
            volume = np.clip(volume, vmin, vmax)
            volume = (volume - vmin) / (vmax - vmin)  # → 0 ~ 1
            volume = volume * 2 - 1                   # → -1 ~ 1
            volume = volume.astype(np.float32)

            # NIfTI 저장
            nifti_img = nib.Nifti1Image(volume, affine=np.eye(4))
            save_path = os.path.join(folder_path, f"{patient_id}.nii.gz")
            nib.save(nifti_img, save_path)

0100.Ischemic Stroke_DWI:  55%|█████▌    | 731/1325 [16:56<13:46,  1.39s/it]  


KeyboardInterrupt: 

In [4]:
data_dir + class_list[k] + '/*MR Diffusion/*' + save_class_list1[j] + '*.dcm'

'../../data/stroke_2023/**/1.원천데이터/0100.Ischemic Stroke/*MR Diffusion/*DWI*.dcm'

In [5]:
import shutil
shutil.rmtree('../../data/Ischemic_stroke_mri/')
shutil.rmtree('../../data/stroke_MRI_preprocessed/')
shutil.rmtree('../../data/2D_MRI/')

KeyboardInterrupt: 

In [11]:
save_class_list=['ADC','DWI']
data_dir='../../data/stroke_2023/**/2.라벨링데이터/'
save_dir='../../data/stroke_mri/label/'

In [12]:
for j in range(len(save_class_list)):
    data_list=glob(data_dir+'0100.Ischemic Stroke'+'/*MR Diffusion/*'+save_class_list[j]+'*.json')
    for i in tqdm(range(len(data_list))):
        with open(data_list[i], 'r') as f:
            json_data = json.load(f)
        label=np.array(json_data["Ischemic Core"])*255
        folder_split=os.path.basename(data_list[i]).split('_')
        folder_name=folder_split[0]+'_'+folder_split[1]
        folder_path=save_dir+save_class_list[j]+'/'+folder_name+'/'
        create_dir(folder_path)
        Image.fromarray(label.astype(np.uint8)).save(folder_path+os.path.splitext(os.path.basename(data_list[i]))[0].split('_')[-1]+'.png')

100%|██████████| 81624/81624 [27:34<00:00, 49.33it/s]  
